In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#on the first level of filtering, I want to get rid of identical networks

#on the second level of filtering, I want to get rid of networks in which
#all interchain Hbond acceptors are also satisfied by intrachain Hbonds
#I think I have to worry less about Hbond donors

In [5]:
from glob import glob
len(glob("/net/scratch/rdkibler/201223_rotor_nmp/*/*.pdb"))

15500

In [6]:
from glob import glob
pdbs = glob("/net/scratch/rdkibler/201224_rotor_hbnet/*/*/*.pdb")

In [4]:
len(pdbs)

15692

In [7]:
len(pdbs)

15692

In [6]:
get_hbnet_comments = False

class NetworkDef:
    def __init__(self,line):
        self.netid,self.residues_str,self.size,self.score,self.num_hbonds,self.percent_hbond_capacity,self.num_unsat_Hpol,self.interf_hbs,self.helics_contacted = line.strip().split()
        self.score = float(self.score)
        self.size = int(self.size)
        self.num_hbonds = int(self.num_hbonds)
        self.percent_hbond_capacity = float(self.percent_hbond_capacity)
        self.num_unsat_Hpol = int(self.num_unsat_Hpol)
        self.interf_hbs = int(self.interf_hbs)
        self.helics_contacted = int(self.helics_contacted)
    
    def __repr__(self):
        return "\t".join([self.netid,self.residues_str,str(self.size),str(self.score),str(self.num_hbonds),str(self.percent_hbond_capacity),str(self.num_unsat_Hpol),str(self.interf_hbs),str(self.helics_contacted)])
    
    def __eq__(self,other):
        return self.residues_str == other.residues_str
    
    def __ne__(self,other):
        return not self == other
    
    def __lt__(self,other):
        return float(self.score) < float(other.score)
    
    def __gt__(self,other):
        return float(self.score) > float(other.score)
    
    def __le__(self,other):
        return not self > other
    
    def __ge__(self,other):
        return not self < other
    
    def __hash__(self):
        return hash(self.residues_str)
    
designs_nets = {}
    

per_design_required_interchain_hbonds = []
per_design_network_defs = []


import tqdm.notebook as tqdm

pbar = tqdm.tqdm(pdbs,total=len(pdbs))
for pdb in pbar:
    max_num_resis_per_chain = 0
    network_defs = []
    hbond_set = set()
    current_hbnet = ""
    with open(pdb,'r') as f:
        
        res_chain_dict = {}
        
        #print(pdb)
        for line in f:
            #print(line,end="")
            if line.startswith("ATOM"):
                max_num_resis_per_chain = int(line.strip().split()[5])
                chain = line[21]
                resn = int(line[22:26])
                res_chain_dict[resn] = chain
                

            #print(line,end="")
            if line == "##Begin comments##\n":
                get_hbnet_comments = True
            if line.startswith("##End comments##"):
                get_hbnet_comments = False

            if get_hbnet_comments:
                #print(line)
                if line.startswith("##network"):
                    network_defs.append(NetworkDef(line))
                    current_hbnet = line.split()[0]
                    #print(line,end="")
                    continue
                if line.startswith("#AtomPair"):
                    sline = line.strip().split()
                    donor_atom = sline[3]
                    donor_resi = int(sline[4]) % max_num_resis_per_chain
                    acceptor_atom = sline[1]
                    acceptor_resi = int(sline[2]) % max_num_resis_per_chain
                    #print(f"hbond atom {donor_atom} at {donor_resi} to atom {acceptor_atom} at {acceptor_resi}")
                    hbond_set.add((donor_atom,donor_resi,acceptor_atom,acceptor_resi,current_hbnet))
                    continue


            if line.startswith("core_pymol_selection select core"):
                sline = line.split()
                core_resis = set([int(resi) for resi in sline[7][:-1].split(",")])
                #print("core_resis",core_resis)
                #print(line)
            if line.startswith("HBNet_NumUnsatHpol"):
                sline = line.split()
                num_unsat_hpol = int(sline[1])
                #print("num_unsat_hpol",num_unsat_hpol)
            if line.startswith("HBNet_Saturation"):
                sline = line.split()
                hbnet_sat = sline[1]
                #print("hbnet_sat",hbnet_sat)
            if line.startswith("HBNet_Score"):
                sline = line.split()
                hbnet_score = sline[1]
                #print("hbnet_score",hbnet_score)
            if line.startswith("spanning_core_hbnets_C"):
                sline = line.split()
                spanning_c = sline[1]
                #print("spanning_c",spanning_c)
            if line.startswith("spanning_core_hbnets_N"):
                sline = line.split()
                spanning_n = sline[1]
                #print("spanning_n",spanning_n)
    #print("num_resis:",max_num_resis_per_chain)

#     print("networks")
#     for network in network_defs:
#         print(network)


    # print("all hbonds:")
    # for donor_atom,donor_resi,acceptor_atom,acceptor_resi,hbond in hbond_set:
    #     print(donor_resi,donor_atom,acceptor_resi,acceptor_atom,hbond)



    # print("fully buried hbonds:")
    # for donor_atom,donor_resi,acceptor_atom,acceptor_resi in hbond_set:
    #     if donor_resi in core_resis and acceptor_resi in core_resis:
    #         print(donor_resi,donor_atom,acceptor_resi,acceptor_atom)

    intrachain_hbonds = []
    buried_interchain_hbonds = []
    #print("intrachain_hbonds")
    for donor_atom,donor_resi,acceptor_atom,acceptor_resi,hbnet in hbond_set:
        if res_chain_dict[donor_resi] != res_chain_dict[acceptor_resi]:
            #print(donor_resi,donor_atom,acceptor_resi,acceptor_atom,hbnet)
            intrachain_hbonds.append((donor_atom,donor_resi,acceptor_atom,acceptor_resi,hbnet))

        #fully buried interchain hbonds
        for donor_atom,donor_resi,acceptor_atom,acceptor_resi,hbnet in hbond_set:
            #print(donor_resi, donor_resi in core_resis,acceptor_resi,acceptor_resi in core_resis)
            if donor_resi in core_resis and acceptor_resi in core_resis:
                #print(donor_resi,donor_atom,acceptor_resi,acceptor_atom)
                #print("HITTT")
                buried_interchain_hbonds.append((donor_atom,donor_resi,acceptor_atom,acceptor_resi,hbnet))

    #now I must determine if I would create a buried unsat if I mismatched these.
    #this is not the best way to check b/c I completely ignore unsatisfied donors but it's a start
    required_interchain_hbonds = []
    for interchain_donor_atom,interchain_donor_resi,interchain_acceptor_atom,interchain_acceptor_resi,interchain_hbnet in buried_interchain_hbonds:
        good = True
        for intrachain_donor_atom,intrachain_donor_resi,intrachain_acceptor_atom,intrachain_acceptor_resi,intrachain_hbnet in intrachain_hbonds:
            if interchain_acceptor_atom == intrachain_acceptor_atom and interchain_acceptor_resi == intrachain_acceptor_resi:
                good = False
                break
        if good:
            required_interchain_hbonds.append((interchain_donor_atom,interchain_donor_resi,interchain_acceptor_atom,interchain_acceptor_resi,interchain_hbnet))

    per_design_required_interchain_hbonds.append(required_interchain_hbonds)
    per_design_network_defs.append(network_defs)

In [7]:
from collections import defaultdict

In [8]:
accepted_nets_designs = defaultdict(lambda:[]) 

for (required_interchain_hbonds,network_defs,pdb) in zip(per_design_required_interchain_hbonds, per_design_network_defs, pdbs):

    
    #print("REQUIRED INTERCHAIN HBONDS")
    #print(pdb,required_interchain_hbonds)
    max_num_unsat_Hpol = 2
    min_percent_hbond_capacity = 0.65
    min_required_interchain_hbonds = 1

    if len(required_interchain_hbonds) >= min_required_interchain_hbonds and sum([net.num_unsat_Hpol for net in network_defs]) <= max_num_unsat_Hpol and all([net.percent_hbond_capacity >= min_percent_hbond_capacity for net in network_defs]):
        important_netids = [x[4] for x in required_interchain_hbonds]
        important_nets = tuple([net for net in network_defs if net.netid in important_netids])
        #designs_nets[pdb] = network_defs
        tiebreaker_score = sum([net.score for net in network_defs])
    #         designs_nets[pdb] = (important_nets,sum_scores_all)        


        accepted_nets_designs[important_nets].append((pdb,tiebreaker_score,important_nets))


In [9]:
len(accepted_nets_designs)

2799

In [10]:
##only 2799 out of 15692 designs passed, or 17.8%

In [11]:
c = 0
for k in accepted_nets_designs.keys():
    c += len(accepted_nets_designs[k])
c

5910

In [12]:
for k in sorted(accepted_nets_designs.keys(), key=lambda x:len(accepted_nets_designs[x])):
    print(len(accepted_nets_designs[k]), k[0].residues_str)

1 A_H_7,A_W_24,B_Q_114,backbone
1 A_H_7,A_H_24,A_Q_37,B_Q_114,backbone
1 A_H_7,A_S_8,A_H_17,B_S_109,B_H_121
1 A_S_7,A_Q_18,A_T_21,B_Q_121,B_Q_122,B_H_157
1 A_S_7,A_E_18,A_T_21,B_Q_121,B_Q_122,B_H_157
1 A_Q_17,A_S_21,A_T_24,B_N_125,B_Q_151
1 A_Q_24,B_W_122,A_S_28,B_S_129,A_Q_37,A_S_27
1 A_H_7,A_T_8,A_Q_17,A_S_21,B_S_109,B_H_160
1 A_H_7,A_T_17,A_H_24,A_Q_27,B_T_109,B_Q_114,B_S_119
1 A_T_7,A_H_21,B_N_109,B_R_118,B_T_122,B_S_151,B_S_160
1 A_Q_20,B_Q_106,B_N_109,B_S_122
1 A_Q_20,B_Q_106,B_N_109
1 A_K_18,A_N_21,B_S_121,B_S_125,B_T_151,B_S_164
1 A_K_18,A_H_21,B_S_121,B_Q_157
1 A_Q_18,A_H_21,B_S_121,B_Q_157
1 A_H_7,A_T_8,A_H_17,A_Q_21,B_S_109,B_S_125,B_Q_160,backbone
1 A_Q_21,B_N_125,B_T_148,B_Q_160,backbone
1 A_Q_21,B_N_125,B_T_148,B_Q_160,B_K_157,A_D_18,backbone
1 A_Q_21,B_N_125,B_T_148,B_K_157,B_Q_160,backbone
1 A_S_7,B_Q_122,B_N_125,B_T_148
1 A_S_17,A_T_20,B_N_121,B_T_122
1 A_T_17,A_S_20,B_N_121,B_T_122,B_N_151,B_S_163
1 A_S_17,A_T_20,B_H_121,B_T_122
1 A_S_8,A_H_24,B_T_108,B_T_109,B_Q_122


In [22]:
keep_top_n = 3

passing = []
for k in accepted_nets_designs.keys():
    print([x[1] for x in sorted(accepted_nets_designs[k],key=lambda x:x[1])[:keep_top_n]])
    passing.extend(sorted(accepted_nets_designs[k],key=lambda x:x[1])[:keep_top_n])
len(passing)

[-0.0900455, 1.80225, 3.19979]
[1.79827, 2.33708]
[-1.48285]
[0.153946]
[3.65673, 7.23731, 8.20786]
[-0.299707, 1.01466, 1.51614]
[1.16168, 1.34107, 1.52572]
[0.248974, 0.359272, 0.457578]
[0.247621, 0.460788, 0.768649]
[1.29599, 1.31295, 1.39888]
[3.1099, 3.18661, 3.29095]
[-0.557021, -0.389325, -0.315552]
[-2.58803, -2.36711, -2.27745]
[6.27827, 6.92576]
[3.01751]
[3.20761, 3.28188, 3.35158]
[-0.0291416, 0.629268, 1.98361]
[-0.469045]
[-0.692238, -0.359051, -0.129232]
[0.604982]
[1.25065, 1.56451]
[2.9818, 5.44629]
[1.74701]
[-0.4377, 0.53277, 1.2604419999999998]
[0.274032, 1.41631, 1.58392]
[1.01469, 1.69847, 1.81485]
[1.30287, 1.53152, 1.86911]
[0.183468, 0.205095, 0.45013]
[0.510469, 0.580228, 0.721703]
[-3.51426, -2.8878, -2.80584]
[5.51543, 10.3606]
[13.6474]
[0.889769, 1.02583, 1.03786]
[0.828936, 1.12957, 1.13012]
[-0.49677, -0.446863, 0.210389]
[1.23479, 2.20391]
[1.16986, 1.49895, 2.2426]
[4.05476]
[1.68717]
[11.6601]
[-2.12402, -1.92081, -1.51042]
[0.6283]
[1.22554]
[2.3073

3815

In [23]:
passing[:6]

[('/net/scratch/rdkibler/201224_rotor_hbnet/0/RC4_20_mini_nm_00152_1.6_31_0001_0038/RC4_20_mini_nm_00152_1.6_31_0001_0038_hb_0001.pdb',
  -0.0900455,
  (##network_1	A_Q_28,B_S_128,B_S_144,B_T_148	4	-0.0900455	5	0.8	0	2	3,)),
 ('/net/scratch/rdkibler/201224_rotor_hbnet/3/RC4_20_mini_nm_00108_1.6_31_0001_0013/RC4_20_mini_nm_00108_1.6_31_0001_0013_hb_0001.pdb',
  1.80225,
  (##network_1	A_Q_28,B_S_128,B_S_144,B_T_148	4	1.80225	5	0.8	0	2	3,)),
 ('/net/scratch/rdkibler/201224_rotor_hbnet/0/RC4_20_mini_nm_00208_1.6_31_0001_0052/RC4_20_mini_nm_00208_1.6_31_0001_0052_hb_0001.pdb',
  3.19979,
  (##network_1	A_Q_28,B_S_128,B_S_144,B_T_148	4	3.19979	5	0.8	0	2	3,)),
 ('/net/scratch/rdkibler/201224_rotor_hbnet/3/RC4_20_mini_nm_00005_1.6_31_0001_0053/RC4_20_mini_nm_00005_1.6_31_0001_0053_hb_0003.pdb',
  1.79827,
  (##network_3	A_Q_28,B_N_125,B_S_128,B_S_144,B_S_148	5	1.79827	6	0.714286	1	2	3,)),
 ('/net/scratch/rdkibler/201224_rotor_hbnet/3/RC4_20_mini_nm_00108_1.6_31_0001_0013/RC4_20_mini_nm_00108_

In [ ]:
import random

In [101]:
for x in random.choices(passing,k=50):
    print(x[0])

/net/scratch/rdkibler/201224_rotor_hbnet/3/RC4_20_mini_nm_00090_1.6_31_0001_0005/RC4_20_mini_nm_00090_1.6_31_0001_0005_hb_0003.pdb
/net/scratch/rdkibler/201224_rotor_hbnet/0/RC4_20_mini_nm_00130_1.6_31_0001_0020/RC4_20_mini_nm_00130_1.6_31_0001_0020_hb_0002.pdb
/net/scratch/rdkibler/201224_rotor_hbnet/1/RC4_20_mini_nm_00169_1.6_31_0001_0001/RC4_20_mini_nm_00169_1.6_31_0001_0001_hb_0003.pdb
/net/scratch/rdkibler/201224_rotor_hbnet/3/RC4_20_mini_nm_00049_1.6_31_0001_0053/RC4_20_mini_nm_00049_1.6_31_0001_0053_hb_0001.pdb
/net/scratch/rdkibler/201224_rotor_hbnet/1/RC4_20_mini_nm_00187_1.6_31_0001_0052/RC4_20_mini_nm_00187_1.6_31_0001_0052_hb_0002.pdb
/net/scratch/rdkibler/201224_rotor_hbnet/0/RC4_20_mini_nm_00089_1.6_31_0001_0003/RC4_20_mini_nm_00089_1.6_31_0001_0003_hb_0003.pdb
/net/scratch/rdkibler/201224_rotor_hbnet/2/RC4_20_mini_nm_00167_1.6_31_0001_0032/RC4_20_mini_nm_00167_1.6_31_0001_0032_hb_0002.pdb
/net/scratch/rdkibler/201224_rotor_hbnet/0/RC4_20_mini_nm_00051_1.6_31_0001_0044/RC

In [24]:
with open("save_passing.txt",'w') as f:
    for pdb,_,_ in passing:

        f.write(pdb+"\n")